In [7]:
import os
import pandas as pd
from bs4 import BeautifulSoup

In [8]:
SCORE_DIR = "Data/Updated_Scores"

In [9]:
box_scores = os.listdir(SCORE_DIR)

In [10]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")] 

In [11]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()
        
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [12]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs={"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    return line_score 

In [13]:
def read_stats (soup, team, stat):
    df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df
    

In [14]:
def read_season_info(soup):
    nav = soup.select("#bottom_nav_container") [0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0] 
    return season

In [15]:
base_cols = None
games = []

for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score["team"])

    summaries = []
    
    for team in teams:
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1, :]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])

        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]

        summary = summary[base_cols]

        summaries.append(summary)

    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, line_score], axis=1)

    game["home"] = [0, 1]
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)

    full_game["season"] = read_season_info(soup)

    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format ="%Y%m%d")

    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    
    if len(games) % 1 == 0:
        print(f"{len(games)} / {len(box_scores)}")
    
    



1 / 6
2 / 6
3 / 6
4 / 6
5 / 6
6 / 6


In [16]:
games_df = pd.concat(games, ignore_index=True)

In [17]:
games_df

mp     mp    fg    fga    fg%    3p   3pa    3p%    ft   fta  ...  \
0   240.0  240.0  42.0   90.0  0.467  11.0  29.0  0.379  19.0  23.0  ...   
1   240.0  240.0  50.0  102.0  0.490  14.0  38.0  0.368   9.0   9.0  ...   
2   240.0  240.0  47.0   86.0  0.547  14.0  36.0  0.389   8.0  10.0  ...   
3   240.0  240.0  45.0   89.0  0.506  10.0  37.0  0.270  12.0  16.0  ...   
4   240.0  240.0  36.0   93.0  0.387  11.0  37.0  0.297  18.0  20.0  ...   
5   240.0  240.0  39.0   79.0  0.494  14.0  35.0  0.400  18.0  22.0  ...   
6   290.0  290.0  45.0  104.0  0.433  22.0  54.0  0.407  11.0  19.0  ...   
7   290.0  290.0  44.0  110.0  0.400  13.0  37.0  0.351  31.0  38.0  ...   
8   240.0  240.0  38.0   95.0  0.400  12.0  35.0  0.343   8.0  13.0  ...   
9   240.0  240.0  46.0   86.0  0.535  15.0  35.0  0.429  10.0  11.0  ...   
10  240.0  240.0  52.0   96.0  0.542  15.0  43.0  0.349  17.0  21.0  ...   
11  240.0  240.0  46.0   98.0  0.469  13.0  35.0  0.371  20.0  30.0  ...   

    tov%_max_opp  usg%_max_opp  ortg_max_opp  drtg_max_opp  team_opp  \
0           17.5          26.8         173.0         122.0       IND   
1          100.0          40.8         136.0         132.0       NOP   
2          100.0          27.8         169.0         125.0       LAC   
3           33.3          30.1         145.0         124.0       LAL   
4           34.2          34.4         152.0         119.0       MIN   
5           33.3          32.1         139.0         124.0       MEM   
6           29.1          29.1         150.0         113.0       CHI   
7           23.7          25.1         176.0         122.0       CLE   
8           50.0          46.9         300.0         111.0       DEN   
9           33.8          29.0         130.0         127.0       SAC   
10          50.0          30.5         143.0         135.0       TOR   
11          35.0          61.9         208.0         126.0       DAL   

    total_opp  home_opp  season       date    won  
0         123         1    2024 2024-02-28  False  
1         114         0    2024 2024-02-28   True  
2         112         1    2024 2024-02-28   True  
3         116         0    2024 2024-02-28  False  
4         110         1    2024 2024-02-28  False  
5         101         0    2024 2024-02-28   True  
6         132         1    2024 2024-02-28  False  
7         123         0    2024 2024-02-28   True  
8         117         1    2024 2024-02-28  False  
9          96         0    2024 2024-02-28   True  
10        125         1    2024 2024-02-28   True  
11        136         0    2024 2024-02-28  False  

[12 rows x 150 columns]

In [18]:
games_df.to_csv('updated_nba_boxscores.csv')